In [ ]:
def get_state():
    x, y = st.session_state.agent_pos
    return np.array([x / gw.GRID_WIDTH, y / gw.GRID_HEIGHT])  # Normalisé


In [ ]:

def train_dqn(model, env, episodes=1000, max_steps=50, gamma=0.95, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.995, alpha=0.01):
    memory = deque(maxlen=2000)  # Replay Buffer

    for ep in range(episodes):
        env.reset_position( placeholder)
        state = get_state()
        total_reward = 0

        for step in range(max_steps):
            # --- Choix de l'action (epsilon-greedy) ---
            if np.random.rand() < epsilon:
                action_idx = np.random.randint(4)
            else:
                q_values = model.predict(state.reshape(1, -1), verbose=0)
                action_idx = np.argmax(q_values[0])

            action = ACTIONS[action_idx]
            
            # --- Appliquer action ---
            time.sleep(sleep_time)
            agent.move_agent(action)
            x_next, y_next = st.session_state.agent_pos
            reward = st.session_state.R[x_next][y_next]

            next_state = get_state()
            done = (x_next, y_next) == env.goal_pos  # terminal state

            # --- Stocker transition dans le buffer ---
            memory.append((state, action_idx, reward, next_state, done))

            state = next_state
            total_reward += reward

            if done:
                break

        # --- Entraînement par batch ---
        batch_size = min(64, len(memory))
        minibatch = random.sample(memory, batch_size)

        for s, a, r, s_next, d in minibatch:
            target = r
            if not d:
                target += gamma * np.max(model.predict(s_next.reshape(1, -1), verbose=0))
                print("target", target)

            target_f = model.predict(s.reshape(1, -1), verbose=0)
            target_f[0][a] = target  # appliquer l'équation de Bellman

            model.fit(s.reshape(1, -1), target_f, epochs=1, verbose=0)

        # --- Décroissance epsilon ---
        if epsilon > epsilon_min:
            epsilon *= epsilon_decay

        st.sidebar.write(f"Épisode {ep+1}, Récompense totale : {total_reward:.2f}, ε = {epsilon:.3f}")